In [2]:
import numpy as np
from tifffile import imsave
from PIL import Image
import torch
img = Image.open('images/nTracer sample.tif')
h, w = np.shape(img)
nframes = int(img.n_frames/4)

def ReadDataFromTif(nframes, h, w):
    img_r = np.zeros((h, w, nframes))
    img_g = np.zeros((h, w, nframes))
    img_b = np.zeros((h, w, nframes))
    img_a = np.zeros((h, w, nframes))
    for i in range(nframes):
        img.seek(i*4)
        img_r[:, :, i] = np.array(img)
        img.seek(i * 4 + 1)
        img_g[:, :, i] = np.array(img)
        img.seek(i * 4 + 2)
        img_b[:, :, i] = np.array(img)
        img.seek(i * 4 + 3)
        img_a[:, :, i] = np.array(img)
    img_rgba = np.max(np.stack([img_r, img_g, img_b, img_a]), axis=0)
    return img_rgba

#pytorch convolution algorithm
image=ReadDataFromTif(nframes,h,w)
image=image-np.min(image)
image=image/np.max(image)
norm_offset=np.mean(image)
image=image-norm_offset
print(np.max(image),np.min(image))

image_tensor = torch.zeros(1,1,np.shape(image)[0],np.shape(image)[1],np.shape(image)[2])
image_tensor[0,0,:,:,:] = torch.from_numpy(image)
pool = torch.nn.AvgPool3d((4,4,2))
pooled_image_tensor = pool(image_tensor)
pooled_image_tensor = pooled_image_tensor.to('cpu')
pooled_image = np.einsum('ijk->kij', pooled_image_tensor.detach().numpy()[0,0,:,:,:])
imsave('images/pooled_image.tif',pooled_image.astype(float))

0.9307008403049115 -0.06929915969508844
